In [1]:
import seaborn as sns
import metapack as mp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display 
from itertools import chain
from functools import reduce
import libgeohash as gh

%matplotlib inline
sns.set_context('notebook')
mp.jupyter.init()


In [2]:
#pkg = mp.jupyter.open_package()
pkg = mp.jupyter.open_source_package()
pkg

# Demosearch Census Extract
`civicknowledge.com-census-demosearch-1.1.1` Last Update: 2021-01-28T17:39:06

_Extract of select census variables for demographic radius searches_




 

## Contacts

* **Wrangler** [Eric Busboom](mailto:eric@civicknowledge.com), [Civic Knowledge](http://civicknowledge.com)

## Resources

* ** [combined](notebooks/Combine.ipynb#df)**. Tract datasets combined into a single frame
* ** [agg_map](notebooks/Combine.ipynb#agg_map)**. Description of which columns were aggregated together

## References
<ul><li> <strong>B01001</string>, <em>census://2019/5/US/tract/B01001</em>. Sex by age</li><li> <strong>B01002</string>, <em>census://2019/5/US/tract/B01002</em>. Median Age by Sex</li><li> <strong>B11009</string>, <em>census://2018/5/US/tract/B11009</em>. Coupled Households by Type</li><li> <strong>B26105</string>, <em>census://2018/5/US/tract/B26105</em>. Group Quarters Type (3 Types) by School Enrollment</li><li> <strong>B15001</string>, <em>census://2019/5/US/tract/B15001</em>. Sex by Age by Educational Attainment</li><ul>

In [3]:
frames = [r.dataframe().drop(columns=['stusab', 'county', 'name']) for r in pkg.references()]

In [4]:
# Create the full column map

def col_f(v):
    return not v[0].endswith('_m90') and not v[0] in ('geoid','stusab', 'county','name')

def munge(v):
    return v.title() \
               .replace('Partner Households By Sex Of Partner  - Households  - Total  -', '') \
               .replace('Total Population  - Total  - ', '') \
               .replace(' Total Population  - Total', 'Total Population') \
               .replace('  - ', ', ')[11:].strip()


kv = list(filter(col_f,chain(*[ list(e for e in e.title_map.items()) for e in frames])))

cm = { k:munge(v) for k, v in kv}


In [5]:
df = reduce(lambda left, right: left.join(right), frames[1:], frames[0])
m90_col = [c for c in df.columns if c.endswith('m90')]
df.drop(columns=m90_col, inplace=True)

In [6]:
aggregates = {
    'male_u18':['b01001_003','b01001_004','b01001_005','b01001_006'],
    'female_u18':['b01001_027','b01001_028','b01001_029','b01001_030'],
    'male_18_40': ['b01001_007', 'b01001_008', 'b01001_009', 'b01001_010', 'b01001_011', 'b01001_012', 'b01001_013'],
    'female_18_40': ['b01001_031', 'b01001_032', 'b01001_033', 'b01001_034', 'b01001_035', 'b01001_036', 'b01001_037'],
    'senior': ['b01001_020', 'b01001_021', 'b01001_022', 'b01001_023', 'b01001_024', 'b01001_025', 
               'b01001_044', 'b01001_045', 'b01001_046', 'b01001_047', 'b01001_048', 'b01001_049'],
   
    'male_18_44_college': ['b15001_009', 'b15001_010', 'b15001_017', 'b15001_018', 'b15001_025', 'b15001_026'],
    'female_18_44_college': ['b15001_050', 'b15001_051', 'b15001_058', 'b15001_059', 'b15001_066', 'b15001_067']
}
#

In [7]:
#doc = mp.open_package(pkg.ref) # Re-open in case it has changed since loaded in this notebook
#print([ f'{c.name}' for c in doc.resource('combined').schema_term.find('Table.Column') if c.get_value('sum') == 'x'])


In [8]:
rows = []
for acol, scols in aggregates.items():
    df[acol] = df.loc[:,scols].sum(axis=1)

    for c in scols:
        rows.append( (acol, c, cm[c.upper()]))
    
agg_map = pd.DataFrame(rows, columns=['agg_column', 'source_col','description'])
agg_map

,agg_column,source_col,description
0,male_u18,b01001_003,"Male, Under 5 Years"
1,male_u18,b01001_004,"Male, 5 To 9 Years"
2,male_u18,b01001_005,"Male, 10 To 14 Years"
3,male_u18,b01001_006,"Male, 15 To 17 Years"
4,female_u18,b01001_027,"Female, Under 5 Years"
5,female_u18,b01001_028,"Female, 5 To 9 Years"
6,female_u18,b01001_029,"Female, 10 To 14 Years"
7,female_u18,b01001_030,"Female, 15 To 17 Years"
8,male_18_40,b01001_007,"Male, 18 And 19 Years"
9,male_18_40,b01001_008,"Male, 20 Years"


In [9]:
def get_columns(pkg):
    pkg = mp.open_package(pkg.ref) # Re-open in case it has changed since loaded in this notebook
    return [ e['name'] for e in pkg.resource('combined').columns()]
    
def update_schema(pkg):
    
    pkg = mp.open_package(pkg.ref) # Re-open in case it has changed since loaded in this notebook
    
    for c in pkg.resource('combined').schema_term.find('Table.Column'):
        if not c.description:
            c.description = cm.get(c.name.upper())
    
    pkg.write()
            
#update_schema(pkg)


In [17]:
doc = mp.open_package(pkg.ref) # Re-open in case it has changed since loaded in this notebook

final_cols = get_columns(doc) #+list(aggregates.keys()) # reset the columns to the ones that are in the schema
final_cols.remove('geoid')

In [18]:
df = df[final_cols]
df.head()

,b01001_001,b01001_002,b01001_026,b11009_001,b11009_002,b11009_007,b15001_001,b15001_002,b15001_043,male_u18,female_u18,male_18_40,female_18_40,senior,male_18_44_college,female_18_44_college
geoid,,,,,,,,,,,,,,,,
14000US01001020100,1993,907,1086,765,15,750,1579,722,857,185,229,292,311,264,70,53
14000US01001020200,1959,1058,901,719,21,698,1540,769,771,289,130,317,226,284,41,49
14000US01001020300,3507,1731,1776,1296,83,1213,2700,1310,1390,421,386,561,497,562,49,152
14000US01001020400,3878,1949,1929,1639,49,1590,3166,1581,1585,368,344,560,458,949,264,166
14000US01001020500,10596,5256,5340,4174,27,4147,7918,3798,4120,1458,1220,1669,1706,1463,607,793


In [19]:
final_cols

['b01001_001',
 'b01001_002',
 'b01001_026',
 'b11009_001',
 'b11009_002',
 'b11009_007',
 'b15001_001',
 'b15001_002',
 'b15001_043',
 'male_u18',
 'female_u18',
 'male_18_40',
 'female_18_40',
 'senior',
 'male_18_44_college',
 'female_18_44_college']